In [1]:
from __future__ import division, absolute_import, print_function

import sys
import os
import pickle
import numpy as np
import random

import h5py
from sklearn import metrics 
from collections import Counter
from glob import glob
import gc
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tables import *

from keras import backend as K 
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.backend import manual_variable_initialization 
from keras.models import load_model, Model
from keras.layers import Dense, concatenate, Flatten, Conv1D, BatchNormalization, Input, Dropout
from keras.optimizers import Adam
from keras.callbacks import CSVLogger, ModelCheckpoint, TerminateOnNaN
#import keras

#root
absPath = '/home/angela3/imbalance_pcm_benchmark/'
sys.path.insert(0, absPath)


from src.model_functions import *
from src.Target import Target
from src.postproc_auxiliar_functions import *

os.environ['PYTHONHASHSEED'] = '0' 
np.random.seed(8)
random.seed(8)
tf.random.set_seed(8)

Using TensorFlow backend.


In [2]:
absPath

'/home/angela3/imbalance_pcm_benchmark/'

In [3]:
nfolds = 10
batch_size = 128
epochss = 100
type_padding_prot = "pre_padding"
protein_type = "GPCRs" #"kinases"

In [4]:
#Loading maximum lengths of proteins and compounds
with open("".join((absPath, 'data/prot_max_len.pickle')), "rb") as input_file:
    max_len_prot = pickle.load(input_file)
#Defining protein dictionary    
instarget = Target("AAA")
prot_dict = instarget.predefining_dict()

In [5]:
learning_rate = 5e-4
decay_rate = learning_rate/epochss
adamm = Adam(lr=learning_rate, beta_1=0.1, beta_2=0.001, epsilon=1e-08, decay=decay_rate)

In [6]:
# LEFT BLOCK (to analyse amino acid sequences)
input_seq = Input(shape=(max_len_prot, len(prot_dict)), dtype='float32')
conv_seq = Conv1D(filters=64, padding='same', strides=1, kernel_size=3, activation='relu')(input_seq)
dropout_1 = Dropout(0.4)(conv_seq)
flatten_seq = Flatten()(dropout_1)#(dense_seq)
dense_seq_2 = Dense(50)(flatten_seq)
dropout_2 = Dropout(0.4)(dense_seq_2)

#RIGHT BRANCH (to analyse fingerprints)
input_fps = Input(shape=(881,), dtype='float32')
dense_fps = Dense(50)(input_fps)
dropout_3 = Dropout(0.4)(dense_fps)
#bn_3 =  BatchNormalization()(dense_fps)#(dense_seq_2)#(conv_seq)


#MERGE BOTH BRANCHES
main_merged = concatenate([dropout_2, dropout_3],axis=1)#([dense_seq_2, dense_fps], axis=1)

main_dense = Dense(2, activation='softmax')(main_merged)

#build and compile model
model = Model(inputs=[input_seq, input_fps], outputs=[main_dense])
model.compile(loss='categorical_crossentropy', optimizer = adamm, metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1499, 26)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1499, 64)     5056        input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1499, 64)     0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 95936)        0           dropout_1[0][0]                  
____________________________________________________________________________________________

In [7]:
#file_h5 = open_file("".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold),
#                                 "/compounds_activity.h5")), "w")

#pickle_filename = "".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold), 
#                               "/splitting_lists/", column_name, "_list.pickle"))

In [ ]:
for fold in range(nfolds): 
    print("Fold:", str(fold))

    file_h5 = "".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold),
                                 "/compounds_activity.h5"))
    f = h5py.File(file_h5, 'r')
    group = '/activity'
    table = "prot_comp"
    ##### TEST
    file_test = "".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold),
                                 "/compounds_activity_test.h5"))
    f_test = h5py.File(file_test, 'r')
    
    file_list = "".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold), 
                         "/splitting_lists/splitting_", str(fold),"_list.pickle"))
    with open(file_list, "rb") as input_file:
        splitting_list = pickle.load(input_file)
    
    splitting_list[0].sort()
    splitting_list[1].sort()
    test_indices = range(len(f_test[group][table]))
    #splitting_list[2].sort()
    
    #Defining generators
    train_generator = batch_generator_DL(batch_size, f, group, table, splitting_list[0], 
                                     max_len_prot, type_padding_prot=type_padding_prot)
    val_generator = batch_generator_DL(batch_size, f, group, table, splitting_list[1], 
                                     max_len_prot, type_padding_prot=type_padding_prot)
    
    #defining callbacks
    if not os.path.exists("".join((absPath, "data/", protein_type, 
                                   "/semi_resampling/logs/", str(fold), "/"))):
        os.makedirs("".join((absPath, "data/", protein_type, 
                                   "/semi_resampling/logs/", str(fold), "/")))
    
    log_path = "".join((absPath, "data/", protein_type, 
                                   "/semi_resampling/logs/", str(fold), "/training_log.csv"))
    csv_logger = CSVLogger(log_path)

    if not os.path.exists("".join((absPath, "data/", protein_type, 
                                   "/semi_resampling/checkpoint/", str(fold), "/"))):
        os.makedirs("".join((absPath, "data/", protein_type, 
                                   "/semi_resampling/checkpoint/", str(fold), "/")))

    #if there are already files in the folder, it removes them
    r = glob("".join((absPath, "data/", protein_type, 
                                   "/semi_resampling/checkpoint/", str(fold), "/*")))
    for i in r:
        os.remove(i)
   
    terminan = TerminateOnNaN()
    checkpoint_path = "".join((absPath, "data/", protein_type, 
                                   "/semi_resampling/checkpoint/", str(fold),
                               "/weights-improvement-{epoch:03d}-{val_accuracy:.4f}.hdf5"))
    mcheckpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=0, 
                                          save_best_only=True, save_weights_only=False)

    callbacks_list = [csv_logger, terminan, mcheckpoint ]
    print("Training")
    # fitting the model
    history = model.fit_generator(generator=train_generator, 
                              validation_data=val_generator,
                             steps_per_epoch= int(len(splitting_list[0])/batch_size),
                              validation_steps=int(len(splitting_list[1])/batch_size),
                             epochs=epochss,
                             callbacks=callbacks_list,
                             verbose=1)
    #saving history
    if not os.path.exists("".join((absPath, "data/", protein_type, "/semi_resampling/results/", 
                                   str(fold), "/"))):
        os.makedirs("".join((absPath, "data/", protein_type, "/semi_resampling/results/", str(fold), "/")))

    with open("".join((absPath, "data/", protein_type, "/semi_resampling/results/", str(fold), "/history.pickle")), 'wb') as handle:
        pickle.dump(history, handle)
        
    print("Prediction on test data")
    #PROTEINS
    batch_sequences = list(f_test[group][table][test_indices]["sequence"])
    #COMPOUNDS
    batch_compounds = list(f_test[group][table][test_indices]["fingerprint"])
    #LABELS
    batch_y = list(f_test[group][table][test_indices]["label"])
    #processing sequences and compounds
    seqs_onehot = np.asarray(processing_sequences(batch_sequences, max_len_prot, type_padding_prot))
    comps_batch = np.asarray(processing_fingerprints(batch_compounds))
    batch_labels = np.asarray(bin_to_onehot(batch_y))
    
    history_path = "".join((absPath, "data/", protein_type, "/semi_resampling/results/", 
                                   str(fold), "/history.pickle"))
    path_to_confusion = "".join((absPath, "data/", protein_type, "/semi_resampling/results/", 
                                   str(fold), "/"))
    path_to_auc = "".join((absPath, "data/", protein_type, "/semi_resampling/results/", 
                                   str(fold), "/"))
    
    history = plot_history(history_path, "".join((absPath, "data/", protein_type, 
                                                  "/semi_resampling/results/", 
                                   str(fold), "/")))
    path_to_cp = ''.join((absPath, "data/", protein_type, "/semi_resampling/checkpoint/", 
                          str(fold), "/"))

    model, best_path = load_best_model(history, path_to_cp)

    cps_loc = ''.join((absPath, "data/", protein_type, "/semi_resampling/checkpoint/", 
                          str(fold), "/*.hdf5")) 

    #removing the rest of weights
    fileList = glob(cps_loc, recursive=True)
    fileList.remove(best_path)
    if len(fileList) >1:
        for filePath in fileList:
            try:
                os.remove(filePath)
            except OSError:
                print("Error while deleting file")
    
    y_predprob = model.predict([seqs_onehot, comps_batch])
    y_prob = y_predprob[:,1]
    y_pred = y_predprob.argmax(-1)
    y_test = batch_labels.argmax(-1)
    print("Counting predicted: ", Counter(y_pred))
    
    batch_compID_test = list(f_test[group][table][test_indices]["da_comp_id"])
    batch_protID_test = list(f_test[group][table][test_indices]["da_prot_id"])
    
    #confusion matrix
    confusion_matrix(y_test, y_pred, path_to_confusion)
        
    #AUC
    file_auc = ''.join((absPath, "data/", protein_type, "/semi_resampling/results/", 
                                   str(fold), "/AUC.pickle"))
    compute_roc(y_test, y_prob, path_to_auc)
    
    # saving predictions on test set

    predictions_test = pd.DataFrame({"y_test":y_test, "y_prob":y_prob, "y_pred":y_pred, "comp_ID": batch_compID_test,
                                "DeepAffinity Protein ID": batch_protID_test})

    if not os.path.exists("".join((absPath, "data/", protein_type, "/semi_resampling/predictions/", str(fold), "/"))):
        os.makedirs("".join((absPath, "data/", protein_type, "/semi_resampling/predictions/", str(fold), "/")))

    predictions_test.to_csv("".join((absPath, "data/", protein_type, "/semi_resampling/predictions/", str(fold), "/test.csv")))
    f.close()

Fold: 0
Training
Epoch 1/100
1829/1829 [==============================] - 157s 86ms/step - loss: 1.6232 - accuracy: 0.6334 - val_loss: 0.4935 - val_accuracy: 0.6245
Epoch 2/100
1829/1829 [==============================] - 156s 85ms/step - loss: 0.6487 - accuracy: 0.6545 - val_loss: 0.5548 - val_accuracy: 0.6165E
Epoch 3/100
1829/1829 [==============================] - 151s 82ms/step - loss: 0.6497 - accuracy: 0.6580 - val_loss: 0.6475 - val_accuracy: 0.5986
Epoch 4/100
1829/1829 [==============================] - 155s 85ms/step - loss: 0.6487 - accuracy: 0.6600 - val_loss: 0.7728 - val_accuracy: 0.5929
Epoch 5/100
1829/1829 [==============================] - 154s 84ms/step - loss: 0.6452 - accuracy: 0.6624 - val_loss: 0.5269 - val_accuracy: 0.5886
Epoch 6/100
1829/1829 [==============================] - 154s 84ms/step - loss: 0.6436 - accuracy: 0.6627 - val_loss: 0.6971 - val_accuracy: 0.5499
Epoch 7/100
1829/1829 [==============================] - 155s 85ms/step - loss: 0.6464 - accur

1829/1829 [==============================] - 155s 85ms/step - loss: 0.6602 - accuracy: 0.6673 - val_loss: 0.3942 - val_accuracy: 0.6553
Epoch 55/100
1829/1829 [==============================] - 155s 85ms/step - loss: 0.6625 - accuracy: 0.6667 - val_loss: 1.2059 - val_accuracy: 0.6581
Epoch 56/100
1829/1829 [==============================] - 155s 85ms/step - loss: 0.6598 - accuracy: 0.6668 - val_loss: 0.1371 - val_accuracy: 0.6599
Epoch 57/100
1829/1829 [==============================] - 153s 84ms/step - loss: 0.6635 - accuracy: 0.6668 - val_loss: 0.8142 - val_accuracy: 0.6564
Epoch 58/100
1829/1829 [==============================] - 155s 85ms/step - loss: 0.6637 - accuracy: 0.6663 - val_loss: 0.9798 - val_accuracy: 0.6547
Epoch 59/100
1829/1829 [==============================] - 156s 85ms/step - loss: 0.6583 - accuracy: 0.6677 - val_loss: 0.6687 - val_accuracy: 0.6551
Epoch 60/100
1829/1829 [==============================] - 155s 85ms/step - loss: 0.6612 - accuracy: 0.6666 - val_loss: 

1873/1873 [==============================] - 147s 79ms/step - loss: 0.6740 - accuracy: 0.6498 - val_loss: 0.6819 - val_accuracy: 0.6857
Epoch 4/100
1873/1873 [==============================] - 148s 79ms/step - loss: 0.6696 - accuracy: 0.6553 - val_loss: 0.6203 - val_accuracy: 0.6872
Epoch 5/100
1873/1873 [==============================] - 151s 81ms/step - loss: 0.6732 - accuracy: 0.6525 - val_loss: 0.6268 - val_accuracy: 0.6706
Epoch 6/100
1873/1873 [==============================] - 147s 79ms/step - loss: 0.6644 - accuracy: 0.6575 - val_loss: 0.5819 - val_accuracy: 0.6813s: 0.6573 - acc
Epoch 7/100
1873/1873 [==============================] - 151s 81ms/step - loss: 0.6634 - accuracy: 0.6588 - val_loss: 0.6749 - val_accuracy: 0.6839
Epoch 8/100
1873/1873 [==============================] - 148s 79ms/step - loss: 0.6682 - accuracy: 0.6561 - val_loss: 0.5606 - val_accuracy: 0.6842 1:0
Epoch 9/100
1873/1873 [==============================] - 148s 79ms/step - loss: 0.6648 - accuracy: 0.6578

1873/1873 [==============================] - 149s 79ms/step - loss: 0.6554 - accuracy: 0.6664 - val_loss: 0.5582 - val_accuracy: 0.6258A: 1:12 - loss: 0.6374 - ac - ETA:  - ETA: 1:10 - loss: 0.6372 - accuracy:  - ETA: 1:10 - loss: 0.6374 - ac - ETA: 1:09 - loss: 0.63 - ETA: 46s - loss: 0.6475 - accuracy: 0.66 - ETA: 46s - loss: 0. - ETA: 44s - loss: 0.6481 - acc - ETA: 40s - l - ETA: 38s - l - ETA: 29 - ETA:  - ETA: 0s - loss: 0.654
Epoch 48/100
1873/1873 [==============================] - 152s 81ms/step - loss: 0.6536 - accuracy: 0.6676 - val_loss: 0.6242 - val_accuracy: 0.6535
Epoch 49/100
1873/1873 [==============================] - 148s 79ms/step - loss: 0.6587 - accuracy: 0.6652 - val_loss: 0.3824 - val_accuracy: 0.6758
Epoch 50/100
1873/1873 [==============================] - 151s 80ms/step - loss: 0.6598 - accuracy: 0.6670 - val_loss: 0.6809 - val_accuracy: 0.6691
Epoch 51/100
1873/1873 [==============================] - 149s 79ms/step - loss: 0.6598 - accuracy: 0.6647 - val_los

1873/1873 [==============================] - 147s 79ms/step - loss: 0.6510 - accuracy: 0.6682 - val_loss: 0.6279 - val_accuracy: 0.6303
Epoch 83/100
1873/1873 [==============================] - 152s 81ms/step - loss: 0.6556 - accuracy: 0.6660 - val_loss: 0.7711 - val_accuracy: 0.66200.66 - ETA: 50s - loss: 0.6448 - accuracy: 0.67 - ETA: 50s - los - ETA: 48s - loss: 0.6 - ETA: 1s
Epoch 84/100
1873/1873 [==============================] - 150s 80ms/step - loss: 0.6553 - accuracy: 0.6684 - val_loss: 0.4523 - val_accuracy: 0.6680- loss: 0.6505 - accurac - ETA: 34 - ETA: 31 - ETA: 28s - loss: 0.6592 - accuracy: 0.666 - - ETA - ETA: 11s - loss: 0.6591 - accur - ETA: - ETA: 7s - loss: 0.6583 - accura - ETA: 5s - loss: 0.6 - ETA: 4s - loss: 0.6 - ETA: 3s - loss: 0.658 - ETA
Epoch 85/100
1873/1873 [==============================] - 148s 79ms/step - loss: 0.6510 - accuracy: 0.6691 - val_loss: 0.4109 - val_accuracy: 0.6760
Epoch 86/100
1873/1873 [==============================] - 149s 79ms/step - 